In [1]:
# np 实现手写字识别
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
# 读取测试数据，train=True读取训练数据；train=False读取测试数据
train_dataset = datasets.MNIST(root='../data/minist', train=True, transform=data_tf)
test_dataset = datasets.MNIST(root='../data/minist', train=False, transform=data_tf)
train_img = train_dataset.data
train_target = train_dataset.targets
test_img = test_dataset.data
test_target = test_dataset.targets
torch.save(train_img, '../data/minist/train_img.pt')
torch.save(train_target, '../data/minist/train_targets.pt')
torch.save(test_img, '../data/minist/test_img.pt')
torch.save(test_target, '../data/minist/test_targets.pt')


In [2]:
# 数据集
class My_Minist(Dataset):
    def __init__(self, mode):
        assert mode in ['train', 'test']
        self.mode = mode
        if mode == 'train':
            self.img_data = torch.load('../data/minist/train_img.pt')
            self.target = torch.load('../data/minist/train_targets.pt')
        else:
            self.img_data = torch.load('../data/minist/test_img.pt')
            self.target = torch.load('../data/minist/test_targets.pt')
        pass

    def __getitem__(self, idx):
        img = self.img_data[idx].view(-1)
        img = img / 255
        target = self.target[idx]
        target_onehot = F.one_hot(target,num_classes = 10)
        if self.mode == 'train':
            return {'img':img, 'target':target_onehot}
        else:
            return {'img':img, 'target':target}

    def __len__(self):
        return self.target.shape[0]

In [3]:
# 模型
class My_Model(nn.Module):
    def __init__(self) -> None:
        super(My_Model, self).__init__()
        self.linear_0 = nn.Linear(28*28, 28*28*2)
        self.linear_1 = nn.Linear(28*28*2, 28*28*2)
        self.linear_2 = nn.Linear(28*28*2, 10)
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, x):
        x = self.linear_0(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        x = self.softmax(x)
        return x

In [4]:
# 损失函数
class My_Loss(nn.Module):
    def __init__(self) -> None:
        super(My_Loss, self).__init__()

    def forward(self, output, target):
        loss = -torch.mean(torch.sum(torch.log(output) * target, dim=-1))
        return loss
        

In [8]:
# 超参数
lr = 0.001
batch_size = 60000
num_workers = 8
epochs=100

In [9]:
train_dataset = My_Minist(mode='train')
test_dataset = My_Minist(mode='test')
train_dataloader = DataLoader(dataset=train_dataset, 
                              batch_size=batch_size, 
                              num_workers=num_workers, 
                              shuffle=True, 
                              drop_last=True,
                              pin_memory=True)
test_dataloader = DataLoader(dataset=test_dataset, 
                              batch_size=10000, 
                              num_workers=0,
                              pin_memory=True)
model = My_Model()
loss_f = My_Loss()
optimizer = torch.optim.SGD(params=model.parameters(),
                        lr=lr)

In [10]:
for epoch in range(epochs):
    for id, data in enumerate(train_dataloader):
        output = model(data['img'])
        loss = loss_f(output, data['target'])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    right = 0
    with torch.no_grad():
        for id, data in enumerate(test_dataloader):
            output = model(data['img'])
            result = torch.argmax(output,dim=-1)
            right = torch.sum(result == data['target'])
    print(right/len(test_dataloader))

tensor(1215.)
tensor(1229.)
tensor(1241.)
tensor(1261.)
tensor(1284.)
tensor(1299.)
tensor(1314.)
tensor(1330.)


KeyboardInterrupt: 